In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone https://github.com/benjaminrike1/social_score_csr_reports.git

Cloning into 'social_score_csr_reports'...
remote: Enumerating objects: 419, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 419 (delta 0), reused 1 (delta 0), pack-reused 415
Receiving objects: 100% (419/419), 1.69 GiB | 17.15 MiB/s, done.
Resolving deltas: 100% (117/117), done.
Checking out files: 100% (156/156), done.


In [3]:
!pip3 install pickle5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 256 kB 34.9 MB/s 


In [4]:
!pip install tensorflow-text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.6 MB 33.1 MB/s 
     |████████████████████████████████| 511.7 MB 6.2 kB/s 
     |████████████████████████████████| 1.6 MB 57.4 MB/s 
     |████████████████████████████████| 438 kB 65.7 MB/s 
     |████████████████████████████████| 5.8 MB 62.8 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Unins

In [5]:
pip install -q tf-models-official==2.7.0

     |████████████████████████████████| 1.8 MB 30.5 MB/s 
     |████████████████████████████████| 92 kB 11.8 MB/s 
     |████████████████████████████████| 43 kB 2.2 MB/s 
     |████████████████████████████████| 352 kB 53.9 MB/s 
     |████████████████████████████████| 1.2 MB 9.5 MB/s 
     |████████████████████████████████| 1.1 MB 54.9 MB/s 
     |████████████████████████████████| 596 kB 26.6 MB/s 
     |████████████████████████████████| 237 kB 77.4 MB/s 
     |████████████████████████████████| 47.8 MB 17.5 MB/s 
     |████████████████████████████████| 99 kB 6.7 MB/s 


In [6]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd
import pickle5 as pickle

In [7]:
model_path = 'social_score_csr_reports/models/no_aug/'
reloaded_model = tf.saved_model.load(model_path)


In [8]:
def print_my_examples(inputs, results):
  print("Probabilities is on the format: neutral, positive, negative")
  result_for_printing = \
    [f'input: {inputs[i]:<30} : \n Probability: Signal: {results[i][0]:.2f}    Noise: {results[i][1]:.2f}'
                         for i in range(len(inputs))]
  print(*result_for_printing, sep='\n')
  print()


examples = [
    'In 2020, we continued to provide a framework of policies, standards, and implementation expectations to guide our supplier business relationships',
    'In 2020, we conducted risk assessments to help us identify and prioritize the most critical issue areas and suppliers. ',
    'We invest in charitable and community causes in all of our markets and our people regularly take part in volunteering activities to support social initiatives in our communities.',
    'As signatories of the Valuable 500 pledge, we are committed to putting disability on the business leadership agenda',
    'Our diversity was reduced this year',
    'Football is a sport',
    'In 2020, 35.6% of leadership positions were held by women, up from 35.5% in 2019. ',
    'Zero fatalities occurred during the year',
    'The launch of our cultural change CARE programme, which will embed ‘the seven health and safety habits’ within our organisation, was another success and', #positive
    '22% of our senior leadership team members are women', #negative?
    'Our Health and Safety Management System', #neutral
    'During the first quarter of 2021, we saw a spike in health and safety incidents',
    'Even though one uses generally the same letter “M” for both martingales and Markov process, these are a priori completely different processes!',
    'Not every process is either a sub- or a supermartingale',
    'This repository is for the exercises, homework and project. The course syllabus can be found here.',
    'Jeg heter Benjamin og skriver på norsk',
    'To establish the communication between the robot and the cloud via Wi-Fi, you need to find the IP address, as shown in Figure 3.',
    'We have drastically reduced our CO2 emission this year as we prepare to meet the latest regulatory demands.'
]
reloaded_results = tf.nn.softmax(reloaded_model(tf.constant(examples)))

print('Results from the model:')
print_my_examples(examples, reloaded_results)

Results from the model:
Probabilities is on the format: neutral, positive, negative
input: In 2020, we continued to provide a framework of policies, standards, and implementation expectations to guide our supplier business relationships : 
 Probability: Signal: 1.00    Noise: 0.00
input: In 2020, we conducted risk assessments to help us identify and prioritize the most critical issue areas and suppliers.  : 
 Probability: Signal: 1.00    Noise: 0.00
input: We invest in charitable and community causes in all of our markets and our people regularly take part in volunteering activities to support social initiatives in our communities. : 
 Probability: Signal: 1.00    Noise: 0.00
input: As signatories of the Valuable 500 pledge, we are committed to putting disability on the business leadership agenda : 
 Probability: Signal: 1.00    Noise: 0.00
input: Our diversity was reduced this year : 
 Probability: Signal: 1.00    Noise: 0.00
input: Football is a sport            : 
 Probability: Sign

In [16]:
filename = 'social_score_csr_reports/all_reports_df.pickle'
with open(filename, 'rb') as fh:
  data = pickle.load(fh)
data.reset_index(inplace=True)
data.drop(labels=['Company'], inplace=True, axis=1)
data.rename({'index':'Company'}, axis=1, inplace=True)
data.head(5)

,Company,sentences_2016,sentences_2017,sentences_2018,sentences_2019,sentences_2020
0,ABNAMRO,"[ABN AMRO Group N.V., Integrated Annual Repo...","[ABN AMRO Bank N.V., ABN AMRO Group N.V., Inte...","[Banking for better, for generations to com...","[Accelerating the sustainability , shift, ABN ...","[ABN AMRO’s business, strategy, performance a..."
1,Amazon,"[“Jeff, what does Day 2 look like?”, That’s a ...","[2, 0, 1, 7, A N N , U A , LR E P O R T, To ou...","[2, 0, 1, 8, A N N , U A , LR E P O R T, To ou...","[ Sustainability: , Thinking Big September 20...",[All In: Staying the Course on Our Commitment...
2,asr,"[2016, Annual Report, a.s.r., Archimedeslaan 1...","[2017, Annual Report , ASR Nederland N.V., Ar...","[Annual Report2018, ASR Nederland N.V., Archim...","[Annual Report2019, ASR Nederland N.V., Archim...","[Ditisdetijd, vandoen., Annual Report2020, ASR..."
3,bam,"[Integrated ReportIntegrated Report 2015 , Ro...","[Colophon, Royal BAM Group nv, Runnenburg 9, 3...","[Royal BAM Group nv, Runnenburg 9, 3981 , AZ B...","[Integrated Report 2019, BAM plants 150,000 tr...","[Integrated Report 2020, Royal BAM Group nv, R..."
4,CocaCola,[Coca‑Cola European Partners plcAnnual Report ...,"[DELIGHTING CUSTOMERS AND CONSUMERS, Coca‑Co...","[GREAT BEVERAGES , GREAT SERVICE , GREAT PEOPL...","[GREAT PEOPLEGREAT SERVICE, GREAT BEVERAGES , ...","[GREAT PEOPLEGREAT SERVICE, GREAT BEVERAGES, 2..."


In [19]:
data.columns.values[1:]

array(['sentences_2016', 'sentences_2017', 'sentences_2018',
       'sentences_2019', 'sentences_2020'], dtype=object)

In [ ]:
import numpy as np
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

scores = [] #saving the individual company scores
columns = data.columns

# iterating over every company
for element in data['Company'].values:
  comp_scores = []
  for report in data.columns.values[1:]:
    # extracting all sentences from the report
    features = data[data['Company']==element][report].values[0] 
    total_sentences = len(features)

    # variables for saving signal
    signal = 0

    # hitting the model with sentences in chunks to avoid RAM overload
    for batch in chunks(features, 128):
        # calculating softmax for every sentence
        results = tf.nn.softmax(reloaded_model(tf.constant(batch)))
        # argmax to extract label
        argmax = tf.math.argmax(results, axis = 1).numpy()

        #counting pos sentences
        signal += np.count_nonzero(argmax==0)
    # assigning total scores realtive to number of sentences
    comp_scores.append(signal)
  scores.append(comp_scores)

In [ ]:
scores = np.asarray(scores)
scores = scores.T

func = lambda x: 1-x

data['scores_2017'] = [func(x) for x in scores[0]]
data['scores_2018'] = [func(x) for x in scores[1]]
data['scores_2019'] = [func(x) for x in scores[2]]
data['scores_2020'] = [func(x) for x in scores[3]]
data['scores_2021'] = [func(x) for x in scores[4]]

In [ ]:
data[['Company', 'scores_2017','scores_2018','scores_2019','scores_2020','scores_2021']].to_csv('drive/MyDrive/ml/social_scores.csv')

In [ ]:
for i,element in enumerate(data['scores_2021']):
  print("%s: %.3f"%(data['Company'].values[i],element))

ABNAMRO: 0.254
AgroFair: 0.290
Amazon: 0.322
asr: 0.280
bam: 0.381
CocaCola: 0.318
Dataflex: 0.321
DS-Smith: 0.293
Essity: 0.364
Facebook: 0.339
Forbo: 0.300
Google: 0.189
GrandVision: 0.467
Grolsch: 0.248
heineken: 0.307
HEMA: 0.401
Henkel: 0.337
HP: 0.332
ING: 0.332
Intel: 0.306
KPN: 0.297
Microsoft: 0.165
NNgroup: 0.319
Optigroup: 0.284
PepsiCo: 0.326
Philips: 0.367
ProcterandGamble: 0.462
Renewi: 0.274
Rockwool: 0.283
Royal-Avebe: 0.368
Shell: 0.189
tarkett: 0.390
Tesla: 0.248
Trivium: 0.233
VanLanschot: 0.413
